In [ ]:
# 公司运营数据表的规整化

#本函数处理的是——公司运营数据表CompanyOperating，转换目的为
    #'TICKER_SYMBOL','year','month 为主码
    # INDIC_NAME_ID为属性，该字段为指标字段名称
        #其中，指标名称ID(INDIC_NAME_ID)为新生成的
        #生成方法：指标英文名称（长短不一）编码后加上COMP前缀
    # VALUE为值
    
# 公司运营数据表CompanyOperating的字段及含义
    # SECURITY_ID证券代码
    # TICKER_SYMBOL交易代码	
    # INDIC_NAME_CN 指标	
    # END_DATE 报告期
    # VALUE值
    # UNIT_CD单位

In [ ]:
PATH_COMPANYOPERATING_FILE = 'Data/CompanyOperating.xlsx'
DATETIME_UNITS="QUARTER"


# 所需调用函数

In [ ]:
#关于addDataTimeFeatures见【2.函数addDataTimeFeatures】

def addDataTimeFeatures(df_s,date_column,DATETIME_UNITS):
    "依据日期格式生成年月季度半年所在的月"
    df_s['year'] = df_s[date_column].map(lambda x:x.year)
    df_st = pd.DataFrame({'MONTH':df_s[date_column].astype(str).apply(lambda x:x[5:7])})
    df_dict = pd.DataFrame({'MONTH':['01','02','03','04','05','06','07','08','09','10','11','12'],
                            'QUARTER':['03','03','03','06','06','06','09','09','09','12','12','12'],
                           'HALF':['06','06','06','06','06','06','12','12','12','12','12','12'],
                           'YEAR':['12','12','12','12','12','12','12','12','12','12','12','12']})
    df_output = pd.merge(df_st,df_dict,how='left')
    df_s = df_s.reset_index()
    df_s['month'] = df_output[DATETIME_UNITS]
    return(df_s)

# 读取数据

![image.png](Images/CompanyOp.png)

In [ ]:
#功能：产生公司数据
#思路：通过读取公司经营数据，取相应列、按周期匹配出年月、按求和汇总、通过行转列等操作，产生公司数据
import pandas as pd
df_comp = pd.DataFrame(pd.read_excel(PATH_COMPANYOPERATING_FILE,sheet_name=['EN'],
                                     converters={'TICKER_SYMBOL':str,'PARTY_ID':str})['EN'])
    #此处'EN'为原数据表中的sheets名称
    
    
df_comp.head()  
    # SECURITY_ID证券代码
    # TICKER_SYMBOL交易代码	
    # INDIC_NAME_CN  指标中文	
    # INDIC_NAME_EN  指标英文
    # END_DATE 报告期
    # VALUE值
    # UNIT_CD单位

In [ ]:
 #取相应列名
df_comp.columns = [x.lstrip().rstrip() for x in df_comp.columns]
df_comp.columns  

# 新增year和month列

In [ ]:
#按周期新增year列和Month列
df_comp = addDataTimeFeatures(df_comp,'END_DATE',DATETIME_UNITS)
df_comp

# 分组统计

In [ ]:
#按TICKER_SYMBOL','INDIC_NAME_EN','year','month'分组统计，并求和
df_comp_g = df_comp.groupby(['TICKER_SYMBOL','INDIC_NAME_EN','year','month'])[['VALUE']].sum().reset_index()
df_comp_g 

In [ ]:
df_comp_g.index,df_comp_g.columns 

In [ ]:
#上一行代码中 df_comp.groupby(['TICKER_SYMBOL','INDIC_NAME_EN','year','month'])[['VALUE']].sum()的含义
df_1 = df_comp.groupby(['TICKER_SYMBOL','INDIC_NAME_EN','year','month'])[['VALUE']].sum()
    #df_1的行索引为多级索引：'TICKER_SYMBOL','INDIC_NAME_EN','year','month'
    #df_1的列索引只有一个，即VALUE
    
print(" df_1.shape=", df_1.shape)
print("type(df_1)=",type(df_1))
print("df_1.index=",df_1.index)
print("df_1.shape",df_1.shape)
print("df_1.columns=",df_1.columns)


In [ ]:
print("df_1.index=",df_1.index[0:5])

In [ ]:
print("df_1.columns=",df_1.columns)

In [ ]:
df_1.reset_index()
    #reset_index：将多级索引处理成同一级索引

# 将指标名称字符串改变为数值类型

In [ ]:
#查看指标名称一列'INDIC_NAME_EN'
df_comp_g['INDIC_NAME_EN']

In [ ]:
#调用pandas.factorize将指标名称改为分类变量
    #Pandas factorize 的功能与 scikit-learn LabelEncoder类似
    #指标名称字段为'INDIC_NAME_EN'
    
f_indic = pd.factorize(df_comp_g['INDIC_NAME_EN'])
f_indic

# 新增指标名称ID一列

In [ ]:
#新增列'INDIC_NAME_ID'
    #取值为：COMP+ 指标名称的数值编号
    
df_comp_g['INDIC_NAME_ID'] ="COMP"+ pd.Series(f_indic[0]).astype(str)
df_comp_g

# 定义新表

In [ ]:
#透视表
    #id为'TICKER_SYMBOL','year','month'
    #列名为INDIC_NAME_ID
    #值为value
    
df_comp_g = df_comp_g.pivot_table(index=['TICKER_SYMBOL','year','month'],
                                  columns=["INDIC_NAME_ID"],
                                  values=["VALUE"]).reset_index()
    #注：.reset_index()的含义为将多级索引改为多个一级索引
    
df_comp_g

In [ ]:
df_comp_g.columns.values

In [ ]:
#处理列名  
df_comp_g.columns = [ ''.join(col) for col in df_comp_g.columns.values]
df_comp_g

In [ ]:
#缺失值补0
df_comp_g = df_comp_g.fillna(0)
df_comp_g.head()